In [ ]:
!pip install --upgrade scikit-learn
import sklearn
print(sklearn.__version__)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error

import pickle

In [ ]:
from google.colab import drive

#drive.flush_and_unmount()

drive.mount('/content/drive')

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Abalone Prediction/Dataset/train.csv')

test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Abalone Prediction/Dataset/test.csv')

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
null_values = train.isnull().sum()
print(null_values)

#preprocessing


In [ ]:

def preprocess(df):

    return df.assign(Sex=df['Sex'].map({'M': 1, 'F': 2, 'I': 0}))


train_encoded = preprocess(train)
test_encoded = preprocess(test)

In [ ]:
train_encoded.head()

In [ ]:
train_encoded.to_csv('/content/drive/My Drive/Colab Notebooks/Abalone Prediction/Dataset/Encoded dataset/train_encoded.csv', index=False);
test_encoded.to_csv('/content/drive/My Drive/Colab Notebooks/Abalone Prediction/Dataset/Encoded dataset/test_encoded.csv', index=False);

Importing preprocessed dataset

In [ ]:

train_encoded = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Abalone Prediction/Dataset/Encoded dataset/train_encoded.csv')
test_encoded = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Abalone Prediction/Dataset/Encoded dataset/test_encoded.csv')


#Descriptive statistics

In [ ]:

summary_stats = train_encoded.describe()

print(summary_stats)

Correlation analysis

In [ ]:

correlation_matrix = train_encoded.corr()
print(correlation_matrix)


In [ ]:
plt.figure(figsize=(12, 8))

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')

plt.show()

#Visualizations

In [ ]:
sns.histplot(train_encoded['Rings'], kde=True)
plt.title('Distribution frequency of rings')
plt.xlabel('Rings')
plt.ylabel('Frequency')
plt.show()


In [ ]:
sns.boxplot(data=train_encoded[['Height', 'Length', 'Diameter', 'Whole weight', 'Shell weight']])
plt.title('Box plot on physical measurements')
plt.show()

In [ ]:
sns.countplot(x='Sex', data=train_encoded)
plt.title('Sex distribution')
plt.xlabel('Sex')
plt.ylabel('Count')
plt.show()


In [ ]:

plt.scatter(train_encoded['Length'], train_encoded['Rings'], alpha=0.5)
plt.title('Length vs. Rings')
plt.xlabel('Length')
plt.ylabel('Rings')
plt.show()


In [ ]:

sns.violinplot(x='Sex', y='Rings', data=train_encoded)
plt.title('Rings distribution by sex category')
plt.show()

In [ ]:


fig, axs = plt.subplots(2, 2, figsize=(12, 10))
sns.histplot(train_encoded['Whole weight'], kde=True, ax=axs[0, 0])
sns.histplot(train_encoded['Whole weight.1'], kde=True, ax=axs[0, 1])
sns.histplot(train_encoded['Whole weight.2'], kde=True, ax=axs[1, 0])
sns.histplot(train_encoded['Shell weight'], kde=True, ax=axs[1, 1])
plt.show()


#Modelling

In [ ]:

X_train = train_encoded.drop(columns=['id', 'Rings'])
y_train = train_encoded['Rings']


In [ ]:

decision_tree = DecisionTreeRegressor(random_state=42)


decision_tree.fit(X_train, y_train)

In [ ]:

train_predictions = decision_tree.predict(X_train)
rmsle_train = mean_squared_log_error(y_train, train_predictions, squared=False)

print(f'RMSLE on training data: {rmsle_train}')

In [ ]:

X_test = test_encoded.drop(columns=['id'])
test_predictions = decision_tree.predict(X_test)


#Exporting

In [ ]:

submission = pd.DataFrame({
    'id': test_encoded['id'],
    'Rings': test_predictions
})


submission.to_csv('/content/drive/My Drive/Colab Notebooks/Abalone Prediction/Submissions/dt_basic.csv', index=False)

print('Submission file created: dt_basic.csv')

In [ ]:

with open('/content/drive/My Drive/Colab Notebooks/Abalone Prediction/Pickle/dt_basic.pkl', 'wb') as file:
    pickle.dump(decision_tree, file)

print('Model saved as: dt_basic.pkl')